# 总需求总供给模型

总需求总供给模型是初级宏观经济学的一个小高峰，它基本按这个顺序发展

* 总收入等于总支出
* 总收入总消费闭环,
* 国民收入决定理论,投资作为外生变量
* IS曲线，产品市场均衡，投资内生化，引入利率
* LM曲线，货币市场均衡，引入货币供给
* IS-LM模型，利率内生化，
* 总需求曲线，实际货币供给内生化，引入价格指数
* 劳动市场均衡，劳动投入量与实际价格内生化
* 生产函数，通过短期控制几个外生变量为不变
* 总供给曲线，得到劳动市场均衡与总收入关系
* 总需求-总供给模型，内生化价格指数。

以此，我们逐步的内生化了一堆变量。从而我们得到了一个这样的静态模型。即给定一组外生变量的值后，就能确定一堆内生变量的值。
外生变量中一个发生变化，一般造成所有内生变量变化，即均衡状态的漂变。有趣的是，即使是这一静态模型的线性版本，都显得
有限难以求解，更不要说动态模型了。

形式化地，模型$model$是一个映射，将一个特定外生变量向量映射为一个内生变量向量

$$
model: g,\bar{k},t,M \to y,c,i,r,P,N,L_1,L_2,f,h,W
$$

其中

外生变量为

* $g$ 是政府投资
* $\bar{k}$ 是资本存量
* $t$ 是定量税
* $M$ 是名义货币供应

内生变量为

* $y$ 为总收入/总支出
* $c$ 为总消费
* $i$ 为总投资
* $r$ 为利率
* $P$ 为价格指数
* $N$ 为劳动力投入
* $L_1$ 为消费式货币需求
* $L_2$ 为投机式货币需求
* $f$或$h$ 为劳动市场均衡时的实际工资
* $W$ 为名义工资

方程的核心是几个基础方程，其中的函数形式是待定的，所以其中携带一些还没提到的参数。尽管是待定的，但应该满足几条基本性质。

* $y=c(y-t)+i(r)+g$ 产品市场均衡
* $\frac{M}{P}=L_1(y)+L_2(r)$ 货币市场均衡
* $f(N)=h(N)=\frac{W}{P}$ 劳动市场均衡
* $y=y(N,\bar{K})$ 短期总量生产函数

我们假设其中的函数形式如下，取最简单的线性函数

$c(y-t)=C_a+C_b(y-t)$ 总需求被扣除税收后的净收入所决定，其中$C_b>0$表示正关系


$i(r)=i_a-i_br $ 总投资与利率成负相关，即$i_b>0$

$L_1(y)=L_{1a}+L_{1b}y $ 消费式货币需求与总收入正相关，即$L_1{1b}>0$


$L_2(r)=L_{2a}-L_{2b}r $ 投机型货币需求与利率负相关，即$L_{2b}>0$

$f(N)=f_a+f_bN $ 劳动需求与供应量成正比,即$f_b>0$

$h(N)=h_a-h_bN $ 劳动需求与供应量成反比，即$h_b>0$

$g(N,\bar{k})=y_a+y_bN\bar{k} $，总产量与劳动投入与资本存量成正比，即$y_b>0$

Amazing,一个结构如此简单又繁琐的模型，让人不禁想起了神经网络。因为对每个内生变量求一次其决定是一场灾难，
我们转而利用计算机代数处理这种并不需要洞见的问题。作为Jupyter notebook,当然用的是`sympy`


In [12]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 12 13:55:20 2016

@author: yiyuezhuo
"""

from sympy import S, Eq, solve,Symbol,simplify

'''arg'''
Ca,Cb,ia,ib,L1a,L1b,L2a,L2b=S('Ca,Cb,ia,ib,L1a,L1b,L2a,L2b'.split(','))
fa,fb,ha,hb,ya,yb=S('fa,fb,ha,hb,ya,yb'.split(','))

'''exog'''
g,kbar,t,M=S('g,kbar,t,M'.split(','))

'''endog'''
y,c,i,r,P,N,L1,L2,f,h,W=S('y,c,i,r,P,N,L1,L2,f,h,W'.split(','))
N=Symbol('N')


'''equations function hypothesis'''
c_y_t=Eq(c,Ca+Cb*(y-t))
i_r=Eq(i,ia-ib*r)
L1_y=Eq(L1,L1a+L1b*y)
L2_r=Eq(L2,L2a-L2b*r)
f_N=Eq(f,fa+fb*N)
h_N=Eq(h,ha-hb*N)
f_h=Eq(f,h)
#y_N_kbar=Eq(y,ya+yb*N*kbar)

#equs_hypo=[c_y_t,i_r,L1_y,L2_r,h_N,y_N_kbar]

'''equations main'''
y_=Eq(y,c+i+g)
P_=Eq(P,M/(L1+L2))
f_=Eq(f,W/P)
y__=Eq(y,ya+yb*N*kbar)


'''IS curve'''

map_y_c_i=solve([c_y_t,y_,i_r],[y,c,i])

'''LM curve'''

map_y_L1_L2=solve([L1_y,L2_r,P_],[y,L1,L2])

'''total demand curve'''

map_y_r=solve([Eq(y,map_y_c_i[y]),Eq(y,map_y_L1_L2[y])],[y,r])

map_y_c_i_={key:map_y_c_i[key].subs({r:map_y_r[r]}) for key in [y,c,i]}
map_y_L1_L2_={key:map_y_L1_L2[key].subs({r:map_y_r[r]}) for key in [y,L1,L2]}

map_y_c_i_L1_L2={}
map_y_c_i_L1_L2.update(map_y_c_i_)
map_y_c_i_L1_L2.update(map_y_L1_L2_)

'''total supply curve'''

map_y_W_N_f_h=solve([f_N,h_N,f_h,f_,y__],[y,W,N,f,h],dict=True)[0]

'''do it!'''

map_y_P=solve([Eq(y,map_y_W_N_f_h[y]),Eq(y,map_y_c_i_L1_L2[y])],[y,P],dict=True)[0]

map_y_P_c_i_N_W_L1_L2_h_f={}
map_y_P_c_i_N_W_L1_L2_h_f.update(map_y_P)
#map_y_P_c_i_N_W_L1_L2_h_f[c]=map_y_c_i_L1_L2[c].subs(map_y_P)
map_y_P_c_i_N_W_L1_L2_h_f.update({key:map_y_c_i_L1_L2[key].subs(map_y_P) for key in [c,i,L1,L2]})
map_y_P_c_i_N_W_L1_L2_h_f.update({key:map_y_W_N_f_h[key].subs(map_y_P) for key in [W,N,f,h]})

map_y_P_c_i_N_W_L1_L2_h_f={key:simplify(value) for key,value in map_y_P_c_i_N_W_L1_L2_h_f.items()}

In [13]:
map_y_P_c_i_N_W_L1_L2_h_f

{y: (-fa*kbar*yb + fb*ya + ha*kbar*yb + hb*ya)/(fb + hb),
 f: (fa*hb + fb*ha)/(fb + hb),
 c: (Ca*fb + Ca*hb - Cb*fa*kbar*yb - Cb*fb*t + Cb*fb*ya + Cb*ha*kbar*yb - Cb*hb*t + Cb*hb*ya)/(fb + hb),
 h: (fa*hb + fb*ha)/(fb + hb),
 i: (-Ca*fb - Ca*hb + Cb*fa*kbar*yb + Cb*fb*t - Cb*fb*ya - Cb*ha*kbar*yb + Cb*hb*t - Cb*hb*ya - fa*kbar*yb - fb*g + fb*ya - g*hb + ha*kbar*yb + hb*ya)/(fb + hb),
 L1: (L1a*fb + L1a*hb - L1b*fa*kbar*yb + L1b*fb*ya + L1b*ha*kbar*yb + L1b*hb*ya)/(fb + hb),
 N: (-fa + ha)/(fb + hb),
 L2: (-Ca*L2b*fb - Ca*L2b*hb + Cb*L2b*fa*kbar*yb + Cb*L2b*fb*t - Cb*L2b*fb*ya - Cb*L2b*ha*kbar*yb + Cb*L2b*hb*t - Cb*L2b*hb*ya + L2a*fb*ib + L2a*hb*ib - L2b*fa*kbar*yb - L2b*fb*g - L2b*fb*ia + L2b*fb*ya - L2b*g*hb + L2b*ha*kbar*yb - L2b*hb*ia + L2b*hb*ya)/(ib*(fb + hb)),
 P: M*ib*(fb + hb)/(-Ca*L2b*fb - Ca*L2b*hb + Cb*L2b*fa*kbar*yb + Cb*L2b*fb*t - Cb*L2b*fb*ya - Cb*L2b*ha*kbar*yb + Cb*L2b*hb*t - Cb*L2b*hb*ya + L1a*fb*ib + L1a*hb*ib - L1b*fa*ib*kbar*yb + L1b*fb*ib*ya + L1b*ha*ib*kbar*yb + L

非常琐碎，也许还是可以简化的，上面虽然已经用`sympy.simplify`简化了一道(相比某些的确还是简化了很多。。)。
另外如此设定参数的值
```
Ca=1000 
Cb=0.8 
ia=10000 
ib=5000 
L1a=100 
L1b=0.1 
L2a=1000 
L2b=500 
fa=0 
fb=1 
ha=1000 
hb=1 
ya=10
yb=1
```

In [9]:
bind={Ca:1000,
Cb:0.8,
ia:10000,
ib:5000,
L1a:100,
L1b:0.1,
L2a:1000,
L2b:500,
fa:0,
fb:1,
ha:1000,
hb:1,
ya:10,
yb:1}

In [14]:
map_y_P_c_i_N_W_L1_L2_h_f[y].subs(bind)

500*kbar + 10

In [15]:
{key:value.subs(bind) for key,value in map_y_P_c_i_N_W_L1_L2_h_f.items()}

{y: 500*kbar + 10,
 f: 500,
 h: 500,
 i: -g + 100.0*kbar + 0.8*t - 998.0,
 W: 5000000*M/(-1000*g + 600000.0*kbar + 800.0*t + 12000.0),
 N: 500,
 c: 400.0*kbar - 0.8*t + 1008.0,
 P: 10000*M/(-1000*g + 600000.0*kbar + 800.0*t + 12000.0),
 L2: -g/10 + 10.0*kbar + 0.08*t - 99.8,
 L1: 50.0*kbar + 101.0}

In [16]:
map_y_r

{y: (L2b*P*(Ca - Cb*t + g + ia) - ib*(L1a*P + L2a*P - M))/(P*(L1b*ib - L2b*(Cb - 1))),
 r: (L1b*P*(Ca - Cb*t + g + ia) - (Cb - 1)*(L1a*P + L2a*P - M))/(P*(L1b*ib - L2b*(Cb - 1)))}

发现了少了利率，专门算一下

In [22]:
map_y_P_c_i_r_N_W_L1_L2_h_f={}
map_y_P_c_i_r_N_W_L1_L2_h_f.update(map_y_P_c_i_N_W_L1_L2_h_f)
map_y_P_c_i_r_N_W_L1_L2_h_f[r]=simplify(map_y_r[r].subs({P:map_y_P_c_i_N_W_L1_L2_h_f[P]})).subs(bind)
map_y_P_c_i_r_N_W_L1_L2_h_f[r]

g/5000 - 0.02*kbar - 0.00016*t + 2.1996

In [24]:
map_bind={key:value.subs(bind) for key,value in map_y_P_c_i_r_N_W_L1_L2_h_f.items()}
map_bind

{i: -g + 100.0*kbar + 0.8*t - 998.0,
 f: 500,
 y: 500*kbar + 10,
 h: 500,
 c: 400.0*kbar - 0.8*t + 1008.0,
 W: 5000000*M/(-1000*g + 600000.0*kbar + 800.0*t + 12000.0),
 N: 500,
 L2: -g/10 + 10.0*kbar + 0.08*t - 99.8,
 P: 10000*M/(-1000*g + 600000.0*kbar + 800.0*t + 12000.0),
 r: g/5000 - 0.02*kbar - 0.00016*t + 2.1996,
 L1: 50.0*kbar + 101.0}

In [30]:
{key:value.subs({g:100,t:110,kbar:1000,M:100}) for key,value in map_bind.items()}

{r: -17.7980000000000,
 i: 98990.0000000000,
 f: 500,
 h: 500,
 y: 500010,
 L1: 50101.0000000000,
 N: 500,
 c: 400920.000000000,
 P: 0.00166666666666667,
 L2: 9899.00000000000,
 W: 0.833333333333333}

利率是负的？这是线性函数的锅，不过姑且让我们假设它会在某个区域里算合理的近似，把`kbar`减10倍看看

In [32]:
{key:value.subs({g:100,t:110,kbar:100,M:100}) for key,value in map_bind.items()}

{r: 0.202000000000000,
 i: 8990.00000000000,
 f: 500,
 h: 500,
 y: 50010,
 L1: 5101.00000000000,
 N: 500,
 c: 40920.0000000000,
 P: 0.0166666666666667,
 L2: 899.000000000000,
 W: 8.33333333333333}